In [356]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import json
import itertools

import shutil
import sys
import os.path

import pyomo.environ as pe
import pyomo.gdp as pyogdp
from pyomo.util.infeasible import log_infeasible_constraints

In [357]:
def txt2dict(filename):  # type in filename with strings! (e.g. 'Auftraege.txt')
    jobs = {}
    index_dict_job = 0
    for line in open(filename, 'r'):
        index_dict_job +=1
        line = line.replace("\'", "\"")
        jobs["Job_%d" %(index_dict_job)] = json.loads(line)
    return jobs

In [358]:
### extract operations data from jobs for solver - NOT FLEXIBLE
def get_opd(jobs, flexible):
    
    flexible = flexible
    I = set() # jobs i
    ni =  {} # number of operations of job i
    Ji = {} # 1: 1,2,3,4, 2:1,2,3,4,5, ... - set of operations of each job i
    O = set() # (i,o),(...),(n,p) - (job-operation) pairs
    K = set() # {1,2,3,4,5,...) - all machines k
    Ko = {} # (i,o): (k1, k2, k3, ...) - machines for one operation
    pre = {} # (i,o): (k,q) - preceeding operations
    pok = {} # (i,o,k): p - processing time job i on machine k
    Oq = {} # {(i,o,k):{(j,q),(l,r),(...)}, (i,o+1,m):{...}}
    
    
    index_operation = 0 # initial value for 1st job loop
    
    ### jobs
    for j, job in enumerate(jobs):
        index_job = int(j+1)
        ## I
        I.add(index_job)
        
        
        ## Ji
        Ji[index_job] = set()
        ## operations    
        for operation in jobs[job]['task']:
            index_operation = int(operation[9:])
            ## ni
            ni[index_job] = index_operation
            ## Ji
            Ji[index_job].add(index_operation)
            op = (index_job, index_operation)
            ## O
            O.add(op)
            ## pre
            if index_operation == 1: # is any item in TASKS already?
                pre[op] = None
            elif index_operation > 1: # is it the first operation of a job?
                pre[op] = (index_job, index_operation - 1)
            
            ## Ko
            Ko[op] = set()
            ## machines    
            for machine_info in jobs[job]['task'][operation][1]:
                index_machine = int(machine_info[0][1:])
                ## K
                K.add(index_machine)
                ## Ko
                Ko[op].add(index_machine)
                ## p
                pok[(index_job, index_operation, index_machine)] = machine_info[1]
                
    for k in K:
        Oq[k] = [(j,q) for (j,q,m) in pok.keys() if m==k]
    OqOriginal = Oq.copy()
    Oq = {}
    for op in O:
        Oq[op] = set()
        for k, oplist in OqOriginal.items():
            for oper in oplist:
                if oper != op and op in oplist:
                    Oq[op].add(oper)
    
    return  I, ni, Ji, O, K, Ko, pre, pok, Oq, OqOriginal

In [359]:
jobs = txt2dict('/home/luca/python/JupyterProjects/pyomo/15x15x10_jobs.txt')
I, ni, Oi, O, M, Mo, pre, tijk, Oq, OqOriginal = get_opd(jobs, flexible = True)

In [509]:
### initializing model    
model = pe.ConcreteModel()

### initializing SETs: I, O, K
## I - set of jobs
model.J = pe.Set(initialize = I, dimen=1)
## O - set of job-operation pairs
model.O = pe.Set(initialize = O, dimen=2)
## K - set of machines
model.M = pe.Set(initialize = M, dimen=1)
## OK - cartesian product model.O * model.K - machine specific tasks with specific process time units
model.OM = pe.Set(initialize = tijk.keys())
## OQ - cartesian product of model.O * model.O - precedence combinations of operations from different jobs on one machine
model.OQK = pe.Set(initialize= model.O * model.O * model.M, dimen=5,
    filter = lambda model, i, j, p, q, k: i < p and (p,q) != (i,j) and (p,q) in Oq[(i,j)] and (i,j) in Oq[(p,q)] and k in Mo[(i,j)] and k in Mo[(p,q)])
# ## TASKORDER - cartesian product of model.O * model.O - every operation  of a job has one operation in it`s job afterwards
# model.PO = pe.Set(initialize = model.O * model.O, dimen=4, 
#     filter = lambda model, i, o, j, q: (i,o) == pre[(j,q)])

    (type: set).  This WILL potentially lead to nondeterministic behavior in
    Pyomo
    (type: set).  This WILL potentially lead to nondeterministic behavior in
    Pyomo


In [510]:
# model.OQK.pprint()

In [511]:
### initializing PARAMs: ni, Ji, Ko, pre, p

# ## ni - number of operations of each job i
# model.ni = pe.Param(model.I, initialize = lambda model, i: ni[i])
## Ji - operations of each job 1
model.Oi = pe.Param(model.J, initialize = lambda model, i: Oi[i])
## Ko - possible machines for each operation
model.Mo = pe.Param(model.O, initialize = lambda model, i, o: Mo[(i,o)])
# # pre - preceding operation of o
# model.pre = pe.Param(model.O, within=Any, default=set(), initialize = lambda model, i, o: pre[(i,o)])
# ## pok - processing times of operation o on machine k
model.tijk = pe.Param(model.OM, default=set(), initialize = lambda model, i, o, k: tijk[(i, o, k)])
# ## Oq - operations that can be processed by the same machine
# model.Oq = pe.Param(model.O, within=Any, default=set(), initialize = lambda model, i, o: Oq[(i,o)])
## upper bound for decision variables
ub = sum(model.tijk[(i, j, k)] for (i,j,k) in model.tijk)
model.ub = pe.Param(initialize = ub)
L = 1000*ub
model.L = pe.Param(initialize = L)

In [512]:
# model.L.pprint()

In [513]:
### create decision variables
model.Xijk = pe.Var(model.OM, domain=pe.Binary) # 1 if operation o is processed by machine k
model.Sijk = pe.Var(model.OM, bounds=(0,ub)) # start time of operation o
model.Cijk = pe.Var(model.OM, bounds=(0,ub)) # completion time of operation o
model.Yijpqk = pe.Var(model.OQK, domain=pe.Binary)
model.Ci = pe.Var(model.J, bounds=(0,ub))
model.Cmax = pe.Var(bounds=(0, ub)) # MAKESPAN: objective value

### objective function
model.objective = pe.Objective(expr = model.Cmax, sense = pe.minimize) # MAKESPAN

In [514]:
model.Xijk.pprint()

Xijk : Size=219, Index=OM
    Key          : Lower : Value : Upper : Fixed : Stale : Domain
       (1, 1, 9) :     0 :  None :     1 : False :  True : Binary
       (1, 2, 7) :     0 :  None :     1 : False :  True : Binary
       (1, 3, 1) :     0 :  None :     1 : False :  True : Binary
       (1, 3, 2) :     0 :  None :     1 : False :  True : Binary
       (1, 4, 3) :     0 :  None :     1 : False :  True : Binary
       (1, 4, 8) :     0 :  None :     1 : False :  True : Binary
       (1, 5, 1) :     0 :  None :     1 : False :  True : Binary
       (1, 5, 3) :     0 :  None :     1 : False :  True : Binary
       (1, 6, 1) :     0 :  None :     1 : False :  True : Binary
       (1, 6, 3) :     0 :  None :     1 : False :  True : Binary
       (1, 7, 3) :     0 :  None :     1 : False :  True : Binary
       (1, 7, 4) :     0 :  None :     1 : False :  True : Binary
       (1, 8, 9) :     0 :  None :     1 : False :  True : Binary
       (1, 9, 1) :     0 :  None :     1 : False :

In [515]:
# def Const01(model, i, j):
#     return sum(model.Xijk[(i,j,k)] for k in model.Ko[(i,j)]) == 1
# model.Const01 = pe.Constraint(model.O, rule=Const01)

In [516]:
def Const01(model, i, j):
    return sum(model.Xijk[i,j,k] for k in Mo[i,j]) == 1
model.Const01 = pe.Constraint(model.O, rule=Const01)

In [517]:
# def Const02(model, i, j):
#     for k in model.Ko[(i,j)]:
#         return model.Sijk[(i,j,k)] + model.Cijk[(i,j,k)] <= model.Xijk[(i,j,k)] * model.L
# model.Const02 = pe.Constraint(model.O, rule=Const02)

In [518]:
def Const02(model, i, j):
    for k in Mo[i,j]:
        return model.Sijk[i,j,k] + model.Cijk[i,j,k] <= model.Xijk[i,j,k] * L
model.Const02 = pe.Constraint(model.O, rule=Const02)

In [519]:
# def Const03(model, i, j):
#     for k in model.Ko[(i,j)]:
#         return model.Cijk[(i,j,k)] >= model.Sijk[(i,j,k)] + model.tijk[(i,j,k)] - (1 - model.Xijk[(i,j,k)]) * model.L
# model.Const03 = pe.Constraint(model.O, rule=Const03)

In [520]:
def Const03(model, i, j):
    for k in Mo[i,j]:
        return model.Cijk[i,j,k] >= model.Sijk[i,j,k] + model.tijk[i,j,k] - (1 - model.Xijk[i,j,k]) * L
model.Const03 = pe.Constraint(model.O, rule=Const03)

In [521]:
# def Const04(model, i, j, p, q, k):
#     if k in model.Ko[(i,j)] and k in model.Ko[(p,q)]:
#         return model.Sijk[(i,j,k)] >= model.Cijk[(p,q,k)] - model.Yijpqk[(i,j,p,q,k)] * model.L
# model.Const04 = pe.Constraint(model. OQK, rule=Const04)

In [522]:
def Const04(model, i, j, p, q, k):
    if k in Mo[i,j] and k in Mo[p,q]:
        return model.Sijk[i,j,k] >= model.Cijk[p,q,k] - model.Yijpqk[i,j,p,q,k] * L
model.Const04 = pe.Constraint(model. OQK, rule=Const04)

In [523]:
# def Const05(model, i, j, p, q, k):
#     if k in model.Ko[(i,j)] and k in model.Ko[(p,q)]:
#         return model.Sijk[(p,q,k)] >= model.Cijk[(i,j,k)] - (1 - model.Yijpqk[(i,j,p,q,k)]) * model.L
# model.Const05 = pe.Constraint(model.OQK, rule=Const05)

In [524]:
def Const05(model, i, j, p, q, k):
    if k in Mo[i,j] and k in Mo[p,q]:
        return model.Sijk[p,q,k] >= model.Cijk[i,j,k] - (1 - model.Yijpqk[i,j,p,q,k]) * L
model.Const05 = pe.Constraint(model.OQK, rule=Const05)

In [525]:
# def Const06(model, i):
#     for j in range(max(model.Oi[i])-1):
#         return sum(model.Sijk[(i,j+2,k)] for k in model.Ko[(i,j+2)]) >= sum(model.Cijk[(i,j+1,k)] for k in model.Ko[(i,j+1)])
# model.Const06 = pe.Constraint(model.J, rule=Const06)   

In [526]:
def Const06(model, i):
    for j in range(max(Oi[i])-1):
        return sum(model.Sijk[i,j+2,k] for k in Mo[i,j+2]) >= sum(model.Cijk[i,j+1,k] for k in Mo[i,j+1])
model.Const06 = pe.Constraint(model.J, rule=Const06)

In [527]:
# def Const07(model, i):
#     return model.Ci[i] >= sum(model.Cijk[i,max(model.Oi[i]), k] for k in model.Ko[(i,max(model.Oi[i]))])
# model.Const07 = pe.Constraint(model.J, rule=Const07)

In [528]:
def Const07(model, i):
    return model.Ci[i] >= sum(model.Cijk[i,max(Oi[i]), k] for k in Mo[(i,max(Oi[i]))])
model.Const07 = pe.Constraint(model.J, rule=Const07)

In [529]:
print(max(Oi[5]))
model.Const07.pprint()

7
Const07 : Size=15, Index=J, Active=True
    Key : Lower : Body                                    : Upper : Active
      1 :  -Inf :     Cijk[1,11,8] + Cijk[1,11,3] - Ci[1] :   0.0 :   True
      2 :  -Inf :                     Cijk[2,6,2] - Ci[2] :   0.0 :   True
      3 :  -Inf :       Cijk[3,4,1] + Cijk[3,4,3] - Ci[3] :   0.0 :   True
      4 :  -Inf :       Cijk[4,8,1] + Cijk[4,8,2] - Ci[4] :   0.0 :   True
      5 :  -Inf :       Cijk[5,7,8] + Cijk[5,7,3] - Ci[5] :   0.0 :   True
      6 :  -Inf :    Cijk[6,14,10] + Cijk[6,14,5] - Ci[6] :   0.0 :   True
      7 :  -Inf :     Cijk[7,11,3] + Cijk[7,11,4] - Ci[7] :   0.0 :   True
      8 :  -Inf :                     Cijk[8,4,2] - Ci[8] :   0.0 :   True
      9 :  -Inf :                     Cijk[9,9,7] - Ci[9] :   0.0 :   True
     10 :  -Inf :  Cijk[10,11,4] + Cijk[10,11,5] - Ci[10] :   0.0 :   True
     11 :  -Inf :  Cijk[11,15,8] + Cijk[11,15,3] - Ci[11] :   0.0 :   True
     12 :  -Inf :   Cijk[12,3,10] + Cijk[12,3,5] - Ci[12] 

In [530]:
def Const08(model, i):
    return model.Cmax >= model.Ci[i]
model.Const08 = pe.Constraint(model.J, rule=Const08)

In [531]:
def solve_model(SOLVER_NAME, TIME_LIMIT):

    solver = pe.SolverFactory(SOLVER_NAME)

    # set time limit for solving, for following solvers the correct formulations to integrate time limit are given below:

    if TIME_LIMIT != None:
        TIME_LIMIT = TIME_LIMIT
        if 'cplex' in SOLVER_NAME:
            solver.options['timelimit'] = TIME_LIMIT
        elif 'glpk' in SOLVER_NAME:         
            solver.options['tmlim'] = TIME_LIMIT
        elif 'gurobi' in SOLVER_NAME:           
            solver.options['TimeLimit'] = TIME_LIMIT
        elif 'xpress' in SOLVER_NAME:
            solver.options['maxtime'] = TIME_LIMIT 

    solver.solve(model)
    results = [{'Job': i,
                'Operation': j,
                'Machine': k,
                'Start': model.Sijk[i,j,k], 
                'Duration': model.tijk[i,j,k], 
                'Finish': model.Sijk[(i,j,k)] + model.tijk[i,j,k]}
               for i,j,k in model.tijk.keys()]
    results = solver.solve(model, tee=True)
    return results

In [532]:
# choose solver
SOLVER_NAME = 'gurobi'

installed_solvers = {'cbc', 'glpk', 'gurobi'}
if SOLVER_NAME in installed_solvers:
    if SOLVER_NAME == 'gurobi':
        os.chdir("/home/luca/python/optsolv/gurobi1000/linux64")
        results = solve_model(SOLVER_NAME, TIME_LIMIT=None) # add time limit like solve_model(SOLVER_NAME, <TIME_LIMIT>)
        df_res = pd.DataFrame(results)
    else:
        os.chdir("/home/luca/coinbrew/pyomo")
        results = solve_model(SOLVER_NAME, TIME_LIMIT=None) # add time limit like solve_model(SOLVER_NAME, <TIME_LIMIT>)
        df_res = pd.DataFrame(results)
        
else:
    print("Solver <",SOLVER_NAME,"> could not be found or is not listed in list <installed_solvers> above")
    print("\n Try following:")
    print("\n      -     Change directory by os.chdir to directory where solver is installed, also try directories above")
    print("\n      -     Check if pyomo solver name is entried in list installed_solvers")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-29
Read LP format model from file /tmp/tmpoom0dj4m.pyomo.lp
Reading time = 0.01 seconds
x3337: 5768 rows, 3337 columns, 17111 nonzeros
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: AMD Ryzen 7 1800X Eight-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 5768 rows, 3337 columns and 17111 nonzeros
Model fingerprint: 0xd74c648b
Variable types: 455 continuous, 2882 integer (2882 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 7e+02]
  RHS range        [1e+00, 7e+05]
Presolve removed 3738 rows and 2135 columns
Presolve time: 0.02s
Presolved: 2030 rows, 1202 columns, 5994 nonzeros
Variable types: 197 continuous, 1005 integer (1005 binary)
Found heuristic solution: objective 45.0000000

Root relaxation: ob

In [ ]:
# print(results)

In [498]:
# model.Sijk.display()
for task, start in model.Sijk.items():
    if pe.value(model.Xijk[(task)]) != 0:
#         if pe.value(start) !=0:
            print(pe.value(model.Xijk[(task)]), task, pe.value(start))

1.0 (1, 1, 9) 10.999999999994543
1.0 (1, 2, 7) 29.999999999996476
1.0 (1, 3, 2) 658.0
1.0 (1, 4, 3) 658.0
1.0 (1, 5, 3) 658.0
1.0 (1, 6, 3) 658.0
1.0 (1, 7, 4) 658.0
1.0 (1, 8, 9) 6.999999999998636
1.0 (1, 9, 3) 658.0
1.0 (1, 10, 4) 658.0
1.0 (1, 11, 8) 0.0
1.0 (2, 1, 2) 32.99999999869999
1.0 (2, 2, 5) 658.0
1.0 (2, 3, 7) 9.999999999996476
1.0 (2, 4, 5) 658.0
1.0 (2, 5, 2) 21.99999999869999
1.0 (2, 6, 2) 0.0
1.0 (3, 1, 3) 658.0
1.0 (3, 2, 5) 658.0
1.0 (3, 3, 5) 658.0
1.0 (3, 4, 1) 2.999999998699991
1.0 (4, 1, 3) 658.0
1.0 (4, 2, 5) 658.0
1.0 (4, 3, 7) 649.0000000000017
1.0 (4, 4, 3) 658.0
1.0 (4, 5, 7) 651.0
1.0 (4, 6, 7) 3.9999999999984084
1.0 (4, 7, 9) 18.999999999994543
1.0 (4, 8, 1) 4.440892098500626e-16
1.0 (5, 1, 7) 24.999999999996476
1.0 (5, 2, 4) 658.0
1.0 (5, 3, 3) 658.0
1.0 (5, 4, 3) 658.0
1.0 (5, 5, 3) 658.0
1.0 (5, 6, 3) 658.0
1.0 (5, 7, 8) 7.999999998699991
1.0 (6, 1, 2) 658.0
1.0 (6, 2, 9) 654.0
1.0 (6, 3, 5) 658.0
1.0 (6, 4, 3) 658.0
1.0 (6, 5, 9) 654.000000000002
1.0 (6

In [ ]:
schedule = pd.DataFrame(results)
list(schedule)